<a href="https://colab.research.google.com/github/hopecris31/Reddit_Transformer/blob/master/Final_Project_with_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate
!pip install google

In [ ]:
import transformers
import math
import torch
from google.colab import drive
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

drive.mount('/content/drive')

checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True, max_length=10)
tokenizer.pad_token = tokenizer.eos_token

In [85]:
from datasets import load_metric

In [61]:
model_path = '/content/drive/MyDrive/NLP/ChatGRT'

model = AutoModelForSequenceClassification.from_pretrained(model_path)

Some weights of the model checkpoint at /content/drive/MyDrive/NLP/ChatGRT were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/NLP/ChatGRT and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [62]:
# Load evaluation dataset
eval_dataset = load_dataset("reddit", split="train[:5%]")
eval_dataset_path = "/content/drive/MyDrive/NLP/eval_dataset"
eval_dataset.save_to_disk(eval_dataset_path)

Saving the dataset (0/2 shards):   0%|          | 0/192416 [00:00<?, ? examples/s]

In [63]:
Tcheckpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True, max_length=10)
model_path = '/content/drive/MyDrive/NLP/ChatGRT'
model = AutoModelForSequenceClassification.from_pretrained(model_path)

Some weights of the model checkpoint at /content/drive/MyDrive/NLP/ChatGRT were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/NLP/ChatGRT and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
def tokenize_function(example):
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(example["subreddit"], example["content"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = eval_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=eval_dataset.column_names)

Map (num_proc=4):   0%|          | 0/192416 [00:00<?, ? examples/s]

In [66]:
# Create a DataLoader for the evaluation dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
eval_dataloader = DataLoader(tokenized_datasets, batch_size=8)

In [67]:
# Evaluate the model
model.eval()
total_loss = 0
total_eval_steps = 0

In [68]:
block_size = tokenizer.model_max_length

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum([ex for ex in examples[k] if isinstance(ex, list)], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [69]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/192416 [00:00<?, ? examples/s]

In [75]:
training_args = TrainingArguments(
    output_dir="output",
    per_device_eval_batch_size=1,  # Set evaluation batch size to 1
)

In [86]:
def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels)
    recall = recall_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

    

In [87]:
trainer = Trainer(
    model=AutoModelForSequenceClassification.from_pretrained(model_path),
    args=training_args,
    train_dataset=lm_datasets,
    eval_dataset=lm_datasets,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at /content/drive/MyDrive/NLP/ChatGRT were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/NLP/ChatGRT and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
eval_results = trainer.evaluate()

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
print(f"Eval loss: {eval_results['eval_loss']:.2f}")
print(f"Accuracy: {eval_results['accuracy']:.2f}")
print(f"Precision: {eval_results['precision']:.2f}")
print(f"Recall: {eval_results['recall']:.2f}")
print(f"F1 Score: {eval_results['f1']:.2f}")

In [ ]:
for batch in eval_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

    total_loss += loss.item()
    total_eval_steps += 1

eval_loss = total_loss / total_eval_steps
perplexity = math.exp(eval_loss)
print(f"Perplexity: {perplexity:.2f}")

In [ ]:
#from datasets import load_dataset

In [ ]:
#raw_datasets = load_dataset("reddit", split="train[:5%]")
#raw_datasets

In [ ]:
#
#tokenized_datasets

In [ ]:

#tokenized_datasets[1]


In [ ]:
#block_size = tokenizer.model_max_length

"""
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum([ex for ex in examples[k] if isinstance(ex, list)], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
"""

In [ ]:
# lm_datasets = tokenized_datasets.map(
#     group_texts,
#     batched=True,
#     batch_size=1000,
#     num_proc=4,
# )

In [ ]:
# lm_datasets

In [ ]:
# tokenizer.decode(lm_datasets[1]["input_ids"])

In [ ]:
# from transformers import TrainingArguments, Trainer

In [ ]:
# model_name = checkpoint.split("/")[-1]
# training_args = TrainingArguments(
#     f"{model_name}-finetuned-reddit",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     push_to_hub=False
# )

In [ ]:
# from google.colab import drive


In [ ]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=lm_datasets,
#     eval_dataset=lm_datasets,
# )

In [ ]:
# trainer.train()

In [ ]:
# drive.mount('/content/drive')
# PATH = '/content/drive/MyDrive/NLP' # [CHANGE ME]
# #'/content/drive/MyDrive/NLP/ChatGR'
# trainer.save_model (PATH + "/ChatGRT")

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:

sample_1_p = tokenized_datasets[300]
#print(sample_1_p['subreddit'], '\n', sample_1_p['content'])

user_input_subreddit = input("Enter Subreddit: ")
user_input_prompt = input("Enter Prompt: ")


input_p = tokenizer(user_input_subreddit, 
                    user_input_prompt, 
                    padding=True,
                    truncation=True, return_tensors='pt')
input_p = input_p.to('cuda')

model(**input_p)